# Import The libraries and dataset

In [1]:
import torch
import torchvision
import pytorch_lightning as pl
import wandb 
import torchvision.transforms as transforms
import torch.nn.functional as F
from torchvision.datasets import MNIST
import os
from torch.utils.data import DataLoader, random_split


In [2]:
!wandb login 80436e2f16d020c8f91e3ef6e2f29a10efb84f9b

wandb: Appending key for api.wandb.ai to your netrc file: /home/ayush/.netrc
Successfully logged in to Weights & Biases!


# Setup Sweep Parameters

In [8]:
sweep_config = {
    'method': 'grid', #grid, random
    'metric': {
      'name': 'val_loss',
      'goal': 'minimize'   
    },
    'parameters': {

        'learning_rate': {
            'values': [0.1, 0.01,0.001]
        },
        'optimizer': {
            'values': ['adam', 'sgd']
        },
        'model':{
            'values':['VGG16','resnet18',]
        }
    }
}
config_defaults = {
        'learning_rate': 0.001,
        'optimizer': 'adam',
        'model' : 'resnet18'
    }
sweep_id = wandb.sweep(sweep_config)

Create sweep with ID: 3ng2esuh
Sweep URL: https://app.wandb.ai/cayush/uncategorized/sweeps/3ng2esuh


In [9]:
class Classifier(pl.LightningModule):

    def __init__(self):
        super(Classifier, self).__init__()
        if wandb.config.model == 'resnet18':
            self.model = torchvision.models.resnet18()
            self.model.fc = torch.nn.Linear(512,2)
        if wandb.config.model == 'VGG16':
            self.model = torchvision.models.vgg16()
            self.model.classifier[6] = torch.nn.Linear(4096,2)      

    
    def forward(self, x):
        x = self.model(x)      

        return x

    def cross_entropy_loss(self, logits, labels):
        return F.cross_entropy(logits, labels)
    
    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        logits = self.forward(x)   # we already defined forward and loss in the lightning module. We'll show the full code next
        loss = self.cross_entropy_loss(logits, y)

        logs = {'train_loss': loss}
        wandb.log(logs)
        return {'loss': loss, 'log': logs}

    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        logits = self.forward(x)
        loss = self.cross_entropy_loss(logits, y)
        return {'val_loss': loss}

    def validation_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        tensorboard_logs = {'val_loss': avg_loss}
        wandb.log(tensorboard_logs)
        return {'avg_val_loss': avg_loss, 'log': tensorboard_logs}
    
    def prepare_data(self):
        # prepare transforms standard to MNIST
        MNIST(os.getcwd(), train=True, download=True)
        MNIST(os.getcwd(), train=False, download=True)

    def train_dataloader(self):
        transform_data = transforms.Compose([transforms.Resize((224,244)),transforms.ToTensor()])
        data = torchvision.datasets.ImageFolder('./dataset', transform= transform_data)
        train_size = int(0.8*len(data))
        test_size = int(len(data) - train_size)


        self.train_dataset, self.test_dataset = torch.utils.data.random_split(data, (train_size, test_size))
        train_loader = torch.utils.data.DataLoader(self.train_dataset,batch_size=16)
        return train_loader

    def val_dataloader(self):
        test_loader = torch.utils.data.DataLoader(self.test_dataset, batch_size=16)
        return test_loader

    
    def test_dataloader(self):
        '''
        test_loader = torch.utils.data.DataLoader(self.test_dataset, batch_size=16)
        return test_loader
        '''

    def configure_optimizers(self):
        # the lightningModule HAS the parameters (remember that we had the __init__ and forward method but we're just not showing it here)
     
        optimizer = torch.optim.Adam(self.parameters(),lr=wandb.config.learning_rate)
        if wandb.config.optimizer == 'sgd':
            optimizer = torch.optim.SGD(self.parameters(),lr=wandb.config.learning_rate)
        #optimizer =  torch.optim.SGD(self.parameters(),lr=0.01)
        return optimizer


In [10]:
def train():
    wandb.init(config=config_defaults)
    model = Classifier()
    model.prepare_data()
    model.train_dataloader()
    trainer = pl.Trainer(max_epochs = 10 )

    trainer.fit(model)

In [11]:
wandb.agent(sweep_id,function=train)

wandb: Agent Starting Run: 9dojkwxu with config:
	learning_rate: 0.1
	model: VGG16
	optimizer: adam
wandb: Agent Started Run: 9dojkwxu


config: {'learning_rate': 0.1, 'model': 'VGG16', 'optimizer': 'adam'}



wandb: Agent Finished Run: 9dojkwxu 

wandb: Agent Starting Run: iaaop8uh with config:
	learning_rate: 0.1
	model: VGG16
	optimizer: sgd
wandb: Agent Started Run: iaaop8uh


config: {'learning_rate': 0.1, 'model': 'VGG16', 'optimizer': 'sgd'}



wandb: Agent Finished Run: iaaop8uh 

wandb: Agent Starting Run: ifbikpv2 with config:
	learning_rate: 0.1
	model: resnet18
	optimizer: adam
wandb: Agent Started Run: ifbikpv2


...


wandb: Agent Finished Run: ade6px0u 

wandb: Agent Starting Run: 80lukt0v with config:
	learning_rate: 0.001
	model: resnet18
	optimizer: adam
wandb: Agent Started Run: 80lukt0v


config: {'learning_rate': 0.001, 'model': 'resnet18', 'optimizer': 'adam'}



wandb: Agent Finished Run: 80lukt0v 

wandb: Agent Starting Run: r6syszaw with config:
	learning_rate: 0.001
	model: resnet18
	optimizer: sgd
wandb: Agent Started Run: r6syszaw


config: {'learning_rate': 0.001, 'model': 'resnet18', 'optimizer': 'sgd'}



wandb: Agent Finished Run: r6syszaw 

